# Exploration

In [2]:
#import nltk
import pandas as pd

In [11]:
df = pd.read_csv("data/labeled_data.csv", sep = ',').drop("Unnamed: 0", axis=1)
df.head()

,count,hate_speech,offensive_language,neither,class,tweet
0,3,0,0,3,2,!!! RT @mayasolovely: As a woman you shouldn't...
1,3,0,3,0,1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...
2,3,0,3,0,1,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...
3,3,0,2,1,1,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...
4,6,0,6,0,1,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...


In [12]:
df.describe()

,count,hate_speech,offensive_language,neither,class
count,24783.000000,24783.000000,24783.000000,24783.000000,24783.000000
mean,3.243473,0.280515,2.413711,0.549247,1.110277
std,0.883060,0.631851,1.399459,1.113299,0.462089
min,3.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,0.000000,2.000000,0.000000,1.000000
50%,3.000000,0.000000,3.000000,0.000000,1.000000
75%,3.000000,0.000000,3.000000,0.000000,1.000000
max,9.000000,7.000000,9.000000,9.000000,2.000000


In [18]:
df.loc[20000]

count                                                                 3
hate_speech                                                           0
offensive_language                                                    3
neither                                                               0
class                                                                 1
tweet                 RT @shananigans__: 50 shades of shut your bitc...
Name: 20000, dtype: object

In [19]:
df["count"].value_counts()

3    22807
6     1571
4      211
9      167
7       27
Name: count, dtype: int64

In [20]:
df["hate_speech"].value_counts()

0    19790
1     3419
2     1251
3      287
4       21
5        7
6        5
7        3
Name: hate_speech, dtype: int64

In [26]:
df[df["hate_speech"] == 7]["tweet"][6171]

'@infidelpamelaLC I\'m going to blame the black man, since they always blame "whitey" I\'m an equal opportunity hater.'

In [27]:
df.loc[6171]

count                                                                 9
hate_speech                                                           7
offensive_language                                                    1
neither                                                               1
class                                                                 0
tweet                 @infidelpamelaLC I'm going to blame the black ...
Name: 6171, dtype: object

## PROVA NLTK

In [31]:
text = df["tweet"][436]
text

'"Don\'t make me make you fall in live with a nigga like meee..." The birds 1&amp;2 are my favorite songs by Weeknd'

In [33]:
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.tokenize import  word_tokenize

In [35]:
tt = TweetTokenizer()

In [39]:
print(tt.tokenize(text.lower()))
print(word_tokenize(text.lower()))

['"', "don't", 'make', 'me', 'make', 'you', 'fall', 'in', 'live', 'with', 'a', 'nigga', 'like', 'meee', '...', '"', 'the', 'birds', '1', '&', '2', 'are', 'my', 'favorite', 'songs', 'by', 'weeknd']
['``', 'do', "n't", 'make', 'me', 'make', 'you', 'fall', 'in', 'live', 'with', 'a', 'nigga', 'like', 'meee', '...', "''", 'the', 'birds', '1', '&', 'amp', ';', '2', 'are', 'my', 'favorite', 'songs', 'by', 'weeknd']


In [ ]:
def preprocessing(text):
    text.lower()
    # cose preprocessing
    return text

In [ ]:
def tokens():
    # cose tokenize (tipo tweet_tockenize)

In [ ]:
# lemmatization ?
# stemming ?
# normalization ?
# n-grams ?

# representation (word2vec - gensim)
# text clustering/classification
# evaluation con confronti
# conclusioni